# Security Aspects of Machine Learning Lab: Side-channel attacks 

In this lab we will experiment with side-channel attacks using the Chipwhisperer Nano. 

## Setting up the hardware

In [ ]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import time
import numpy as np
from tqdm.notebook import trange

In [ ]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER="SS_VER_1_1"
CRYPTO_TARGET = 'TINYAES128C'

In [ ]:
def reset_target(scope):
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)

In [ ]:
scope = cw.scope()
target = cw.target(scope)
prog = cw.programmers.STM32FProgrammer

In [ ]:
scope.default_setup()

# Checking that everything works using a simple program

In [ ]:
# %%bash -s "$PLATFORM"
# cd hw/simpleserial-base/
# make PLATFORM=$1 CRYPTO_TARGET=NONE

In [ ]:
cw.program_target(scope, prog, f"hw/simpleserial-base/simpleserial-base-{PLATFORM}.hex")

In [ ]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
target.simpleserial_write('p', msg)
print(target.simpleserial_read('r', 16))

Above should give ```bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')```

In [ ]:
target.simpleserial_write('k', msg)
print(target.simpleserial_wait_ack())

Above should give ```0```

In [ ]:
scope.arm()
target.simpleserial_write('p', msg)
## fill in the rest...
scope.capture()
trace = scope.get_last_trace()
cw.plot(trace)

Above should give you a plot containing a power trace of 5000 points

# Our first target: a password checker

In [ ]:
def cap_pass_trace(pass_guess):
    pass_guess = pass_guess + "\n"
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [ ]:
def try_password(pw = None):
    reset_target(scope)
    
    num_char = target.in_waiting()
    while num_char > 0:
        print(target.read(num_char))
        time.sleep(0.03)
        num_char = target.in_waiting()
    
    # time.sleep(0.05)
    if pw is None:
        pw = input()
    target.write(f"{pw}\n")
    print("-" * 50)
    print(f" password = {pw} ".center(50, '-'))
    print("-" * 50)
    time.sleep(0.05)
    
    num_char = target.in_waiting()
    while num_char > 0:
        print(target.read(num_char, 10))
        time.sleep(0.03)
        num_char = target.in_waiting()

In [ ]:
# %%bash -s "$PLATFORM" "$SS_VER"
# cd hw/basic-passwdcheck
# make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
# The program is already compiled, you just need to program the target
cw.program_target(scope, prog, f"hw/basic-passwdcheck/basic-passwdcheck-{PLATFORM}.hex")

You can use the ```try_password()``` function to give a password to the program. If the password is correct you will get the message 'Access granted, Welcome!', if the password is incorrect you get the message 'PASSWORD FAIL'

In [ ]:
# Reduce sample size for this target program
scope.adc.samples = 500

In [ ]:
try_password()

You can use the ```cap_pass_trace``` function to capture a power trace corresponding to the given input password.

In [ ]:
trace = cap_pass_trace("a")
cw.plot(trace)

Let's try to figure out what the first character of the password is. To help you out I will give you the following information about the program. 
- To validate the password, the program compares each character of the input password from left to right to the correct password
- Only the following characters are allowed in the password: ```abcdefghijklmnopqrstuvwxyz0123456789``` i.e. digits and lower-case letters
- The program processes a zero byte character ```"\x00"``` just like any other character, but it is certainly not in the password.

HINT: try to see if the power trace for any (first) character looks different from the other ones

In [ ]:
### IMPLEMENT
raise NotImplementedError()

Once you have figured out how to retreive the first character, you can iteratively retrieve all characters of the password. Try to figure out a way to do this automatically, i.e. without having to manually look at plots. 

In [ ]:
### IMPLEMENT
raise NotImplementedError()

# Our second target: AES

In [ ]:
# AES sbox
sbox = np.array([
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
])

# Hamming Weight: nr. of 1's in binary of given int value. e.g. HW[0] = 0, HW[255] = 8
HW = np.array([bin(n).count("1") for n in range(0, 256)])

In [ ]:
scope.default_setup()

In [ ]:
# %%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
# cd hw/simpleserial-aes
# make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
cw.program_target(scope, prog, f"hw/simpleserial-aes/simpleserial-aes-{PLATFORM}.hex")

## The program
This program takes as input a plaintext using the simpleserial protocol: 'p' followed by the 16 byte plaintext, and returns the ciphertext

In [ ]:
# a key - plaintext sampler: uses a fixed key and random plaintexts
ktp = cw.ktp.Basic()
key, pt = ktp.next()
key

In [ ]:
key, pt = ktp.next()
print(f"encrypting {pt.hex()} using key: {key.hex()}")

target.set_key(key)

target.simpleserial_write('p', pt)
ct = target.simpleserial_read('r', 16)
print(f"ciphertext: {ct.hex()}")

Capture traces

In [ ]:
N = 2000

traces = np.zeros((N, scope.adc.samples))
pts = np.zeros((N, 16), dtype=np.uint8)

key, text = ktp.next()

# set the key on the target
target.set_key(key)
print(f"full key: {key.hex()} - {np.frombuffer(key, dtype=np.uint8)}")

In [ ]:
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()
    ### IMPLEMENT
    raise NotImplementedError()
    
    traces[i] = 
    pts[i] = np.frombuffer(text, dtype=np.uint8)


In [ ]:
f = cw.plot(traces[0])
# f *= cw.plot(traces[1])
f

In [ ]:
# In case you want to do POI selection you can use SNR for this implementation
def snr(obs_value, traces):
    """
    Performs signal-to-noise ratio calculation along the columns.
    
    :obs_value: intermediate value for which you want to calculate the SNR, shape = [N,]
    :traces:    side-channel measurements, shape = [N,P]
    
    :returns:   SNR values, shape=[P,]

    Where
    N = Number of traces
    P = Number of points in a trace
    """
    max_val = np.max(obs_value)
    means = [None] * (max_val + 1)
    varss = [None] * (max_val + 1)
    for i in range(max_val + 1):
        means[i] = np.mean(traces[obs_value == i], axis=0)
        varss[i] = np.var(traces[obs_value == i], axis=0)
    means = np.array(means)
    varss = np.array(varss)
    snr = np.zeros_like(means[0])
    u_tot = np.mean(means, axis=0)
    v_tot = np.mean(varss, axis=0)
    for i in range(max_val + 1):
        snr += np.power(means[i]-u_tot, 2)
    snr = snr / v_tot
    return snr

In [ ]:
key_vec = np.zeros(256)

for kguess in trange(256):
    ### IMPLEMENT
    # Either CPA or DPA suffice
    hypotheses = 
    key_vec[kguess] = 

# print(key_vec)
print(np.argmax(key_vec), key[0])

In [ ]:
plt.plot(key_vec)

# Bonus: Fault attacks

Our chipwhisperer Nano has voltage glitching capabilities. This means that we can intentionally mess with the power supply to the chip. Digital hardware needs a particular (stable) amount of voltage to function properly. If this voltage is not provided according to specification, the chip can misbehave in ways, e.g. corrupting values in registers or skipping instructions. Fault attacks can therefore be powerful attacks.

### Glitch Hardware

The ChipWhisperer Nano's glitch setup is pretty simple, it temporarily shorts the microcontroller's voltage supply to ground:

![](https://wiki.newae.com/images/8/82/Glitch-vccglitcher.png)

For the Nano, `Glitch_In` is controlled by 2 parameters:

1. `scope.glitch.ext_offset` - This determines the timing of the fault - The glitch will be inserted roughly `8.3ns * scope.glitch.ext_offset` after the trigger (which occurs at the start of encryption)
1. `scope.glitch.repeat`     - This determines the length of the fault - The glitch will be inserted for roughly `8.3ns * scope.glitch.repeat`

During this lab, we'll be varying these parameters to see if we can get a target to misbehave.

In [ ]:
# simple serial version
SS_VER = 'SS_VER_2_1'

# when you perform fault attacks you may get a lot of error messages
cw.set_all_log_levels(cw.logging.CRITICAL)

# update target to new serial version
target = cw.target(scope, cw.targets.SimpleSerial2)

In [ ]:
# %%bash -s "$PLATFORM" "$SS_VER"
# cd hw/simpleserial-glitch
# make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
cw.program_target(scope, prog, "hw/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM))

In [ ]:
scope.io.clkout = 7.5E6
target.baud = 230400

In [ ]:
def reboot_flush(scope):            
    reset_target(scope)
    #Flush garbage too
    target.flush()

The simple serial program above contains a couple of functionalities which we can call using commands and the simple serial protocol. To get familiar with fault injection we will first look at a very simple program which we can call with the `g` command. It is simply a loop which always counts to 2500, and returns the count value.

```C
for(i=0; i<50; i++){
        for(j=0; j<50; j++){
            cnt++;
        }
    }
```

We want to see if we can get the chip to return a different value by injecting faults in the power supply.

In [ ]:
# Run the program normally
scope.glitch.repeat = 0

reboot_flush(scope)
scope.arm()
target.simpleserial_write("g", bytearray([]))
scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)
if val['valid']:
    print(val)
    print(f"loop output: {int.from_bytes(val['payload'], byteorder='little')}")

Below you can see some code for how to inject a fault: all you have to do is set the `repeat` and `ext_offset` fields of `scope.glitch`. In this case the fault should be too strong, resulting in a reset of the chip.

In [ ]:
# Injecting a fault which will surely crash the chip
scope.glitch.repeat, scope.glitch.ext_offset = (40, 50)

target.flush()
scope.arm()

target.simpleserial_write("g", bytearray([]))
timeout = scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)
print(timeout)
print(val)

We can use a glitch controller to do a search for successful faults

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset"])
gc.display_stats()
# This will display some counters which you can use to track progress when you're looking for faults

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
gc.set_global_step(1)

# ranges of glitch parameters
gc.set_range("repeat", 12, 13)
gc.set_range("ext_offset", 100, 1000)

reboot_flush(scope)

# we want to try each fault a couple of times because success is stochastic
sample_size = 3
for glitch_setting in gc.glitch_values():
    
    raise NotImplementedError("set the repeat and ext_offset of the fault")
    
    successes = 0
    resets = 0
    
    for i in range(sample_size):
        
        raise NotImplementedError("run the program with fault settings")

        # below you should put your logic to determine what the result of the fault was: a reset, normal behaviour, or a successful fault
        if ...:
            gc.add("reset")
            resets += 1

            reboot_flush()
        else:
            if ... is False:
                reboot_flush(scope)
                gc.add("reset")
                resets += 1
            else:
                if ...: #successful fault
                    gc.add("success")
                    successes += 1
                else:
                    gc.add("normal")
    if successes > 0:       
        print("successes = {}, resets = {}, repeat = {}, ext_offset = {}".format(successes, resets, scope.glitch.repeat, scope.glitch.ext_offset))
print("Done glitching")

## Targeting a password checker

Another functionality of the program is a password checker, which we can call with the command `p`. You can find an example of how to use it below.

In [ ]:
scope.glitch.repeat, scope.glitch.ext_offset = (0,0)
pw = bytearray("password", encoding='utf-8')

reboot_flush(scope)
scope.arm()
target.simpleserial_write("p", pw)
ret = scope.capture()
val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)
print(val)
trace = scope.get_last_trace()


The *payload* field of the response indicates if the password is valid. 
- payload = `0` -> password invalid
- payload = `1` -> password is valid.

So unfortunately, the password "password" is invalid. This time, let's not bother trying to find the correct password, instead let's try to inject a fault so the program thinks that it is.

In [ ]:
raise NotImplementedError("Use a similar approach to above to find a fault to bypass the password checker")

# Run below before exiting: disconnect from the hardware

In [ ]:
scope.dis()
target.dis()